In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('sales_info').getOrCreate()

23/12/13 19:59:35 WARN Utils: Your hostname, DESKTOP-0HJMRJF resolves to a loopback address: 127.0.1.1; using 172.19.51.88 instead (on interface eth0)
23/12/13 19:59:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/13 19:59:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [6]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+


In [7]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)


In [18]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+


In [25]:
df.agg({'Company': 'count'}).show()

+--------------+
|count(Company)|
+--------------+
|            12|
+--------------+


In [30]:
df.agg({'Sales': 'sum'}).show()
df.agg({'Sales': 'max'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+


In [31]:
group_data = df.groupBy("Company")

In [38]:
group_data.agg({'Sales': 'max', 'Person':'min'}).show()

+-------+----------+-----------+
|Company|max(Sales)|min(Person)|
+-------+----------+-----------+
|   APPL|     750.0|      Chris|
|     FB|     870.0|       Carl|
|   GOOG|     340.0|    Charlie|
|   MSFT|     600.0|        Amy|
+-------+----------+-----------+


In [39]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [41]:
df.select(countDistinct('Company')).show()

+-----------------------+
|count(DISTINCT Company)|
+-----------------------+
|                      4|
+-----------------------+


In [46]:
df.select('Company').distinct().show()

+-------+
|Company|
+-------+
|   APPL|
|   GOOG|
|     FB|
|   MSFT|
+-------+


In [57]:
df.select(avg('Sales').alias('Avg_sales')).show()
df.select('Sales').show()


+-----------------+
|        Avg_sales|
+-----------------+
|360.5833333333333|
+-----------------+

+-----+
|Sales|
+-----+
|200.0|
|120.0|
|340.0|
|600.0|
|124.0|
|243.0|
|870.0|
|350.0|
|250.0|
|130.0|
|750.0|
|350.0|
+-----+


In [59]:
df.select(stddev('Sales').alias('Sales_stddev')).show()

+------------------+
|      Sales_stddev|
+------------------+
|250.08742410799007|
+------------------+


In [60]:
from pyspark.sql.functions import format_number

In [74]:
sales_std = df.select(stddev('Sales').alias('std'))
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+


In [76]:
sales_std.select(format_number('std', 2).alias('sales_std')).show()

+---------+
|sales_std|
+---------+
|   250.09|
+---------+


In [80]:
df.select('Company').orderBy(df['Sales'].desc()).show()

+-------+
|Company|
+-------+
|     FB|
|   APPL|
|   MSFT|
|     FB|
|   APPL|
|   GOOG|
|   APPL|
|   MSFT|
|   GOOG|
|   APPL|
|   MSFT|
|   GOOG|
+-------+
